In [4]:
import pandas as pd

### Задание 1

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [268]:
ratings = pd.read_csv('ratings.csv')

In [43]:
def rating_class(row):
    '''Классификация рейтинга'''
    if row['rating'] > 4:
        class_ = 'высокий рейтинг'
    elif row['rating'] > 2:
        class_ = 'средний рейтинг'
    else: class_ = 'низкий рейтинг'
    return class_

In [44]:
ratings['class'] = ratings.apply(rating_class, axis=1)

In [46]:
ratings.head()

,userId,movieId,rating,timestamp,class
0,1,1,4.0,964982703,средний рейтинг
1,1,3,4.0,964981247,средний рейтинг
2,1,6,4.0,964982224,средний рейтинг
3,1,47,5.0,964983815,высокий рейтинг
4,1,50,5.0,964982931,высокий рейтинг


### Задание 2
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {

'Центр': ['москва', 'тула', 'ярославль'],

'Северо-Запад': ['петербург', 'псков', 'мурманск'],

'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [272]:
df = pd.read_csv('keywords.csv')

In [269]:
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [270]:
def region_class(query):
    '''Классификация по регионам в зависимости от упоминания города'''
    geo_data_inverted = {vi: k for k, v in geo_data.items() for vi in v }
    for city in geo_data_inverted.keys():
        if city in query['keyword']:
            return geo_data_inverted[city]
    return f'undefined'

In [273]:
df['region'] = df.apply(region_class, axis=1)

In [274]:
df.region.value_counts()

undefined         99253
Центр               387
Северо-Запад        266
Дальний Восток       94
Name: region, dtype: int64

### Задание 3 
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

* для каждой строки пройдите по всем годам списка years
* если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
* если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [258]:
years = [i for i in range(1950,2020)]
movies = pd.read_csv('movies.csv')

In [259]:
def production_year(movie):
    for year in years:
        if str(year) in movie['title']:
            return year
    return 1900

In [260]:
movies['year'] = movies.apply(production_year, axis=1)

In [262]:
df = ratings.merge(movies, on='movieId', how='left')[['rating','year']]

In [265]:
df.groupby('year').agg(['mean','count'])['rating'].sort_values('mean', ascending=False)

,mean,count
year,,
1957,4.039535,215
1954,4.009191,272
1962,3.969466,262
1952,3.953125,96
1972,3.944293,368
...,...,...
2016,3.387261,785
2005,3.359976,2496
1992,3.353555,2011


#### Другой вариант с использованием регулярных выражений

In [242]:
import re
def production_year_regex(movie):
    pattern = r'.+(?P<year>\d{4})'
    if re.match(pattern, movie['title']):
        return re.match(pattern, movie['title'])['year']
    else: return f'undefined'

In [267]:
movies['year_regex'] = movies.apply(production_year_regex, axis=1)
df_re = ratings.merge(movies, on='movieId', how='left')[['rating','year_regex']]
df_re.groupby('year_regex').agg(['mean','count'])['rating'].sort_values('mean', ascending=False)

,mean,count
year_regex,,
1917,4.500000,1
1930,4.205882,17
1921,4.100000,5
1934,4.088235,34
1944,4.043478,92
...,...,...
1996,3.335329,4509
1932,3.333333,24
1903,2.500000,2
